## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_43.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_43.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_당시_나이",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, 
                                                                            Total_y, 
                                                                            test_size=0.2, 
                                                                            random_state=42,
                                                                            stratify=Total_y)

### Total 데이터

optuna

In [7]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 8000),
        'num_leaves': trial.suggest_int('num_leaves', 100, 5000),
        'max_depth': trial.suggest_int('max_depth', 5, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.0005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'subsample': trial.suggest_float('subsample', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 100.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-19 02:25:05,945] A new study created in memory with name: no-name-421376ac-7a2b-4c58-b18e-54fbd6f52578
[I 2025-02-19 02:25:15,696] Trial 0 finished with value: 0.7365230058351814 and parameters: {'n_estimators': 2509, 'num_leaves': 683, 'max_depth': 254, 'learning_rate': 0.4020231085244065, 'min_child_samples': 148, 'subsample': 0.8721667509861754, 'reg_alpha': 73.58392247909619, 'reg_lambda': 45.86046457987526}. Best is trial 0 with value: 0.7365230058351814.
[I 2025-02-19 02:28:31,543] Trial 1 finished with value: 0.7287914670252171 and parameters: {'n_estimators': 3080, 'num_leaves': 3630, 'max_depth': 321, 'learning_rate': 0.008626839802588878, 'min_child_samples': 295, 'subsample': 0.50043582988575, 'reg_alpha': 3.194473416871872, 'reg_lambda': 54.20062100202439}. Best is trial 0 with value: 0.7365230058351814.
[I 2025-02-19 02:28:53,859] Trial 2 finished with value: 0.7361778889206849 and parameters: {'n_estimators': 7913, 'num_leaves': 1790, 'max_depth': 251, 'learnin

Best trial:
  Value: 0.7382621361232027
  Params: 
    n_estimators: 7655
    num_leaves: 2855
    max_depth: 5
    learning_rate: 0.16281911161693016
    min_child_samples: 460
    subsample: 0.38886888090131794
    reg_alpha: 36.93967560025368
    reg_lambda: 96.03211423886586


.